# 百度网盘AI大赛-图像处理挑战赛：文档检测优化赛12名方案

## 比赛介绍
生活中人们使用手机进行文档扫描逐渐成为一件普遍的事情，为了提高人们的使用体验，我们期望通过算法技术去除杂乱的拍摄背景并精准框取文档边缘，选手需要通过深度学习技术训练模型，对给定的真实场景下采集得到的带有拍摄背景的文件图片进行边缘智能识别，并最终输出处理后的扫描结果图片。

## 任务分析
本次任务可以同时看作回归问题和分割问题。

- 回归任务：设计方案回归文档四个拐角点的坐标
- 分割任务：设计方案对图片中文档进行分割

## 技术分析

- 模型选择
    0. [基线](https://aistudio.baidu.com/aistudio/projectdetail/3861946)使用了回归模型，但是经过测试，Unet可以显著优于Resnet+Linear的基线模型。对比基线模型，分割模型大约能够提高5个百分点。因此，本项目使用分割模型处理问题。
    1. 对于Unet/Unet++/Unet3/OCRNet等而言，其训练精度没有产生质变。本项目使用效果最好的OCRNet，但不否认Unet可能会在随机作用下产生更好的结果。

- 数据处理

    0. 由于本次数据中的所有图片均为512*512，因此不需要对图片进行Resize，也不需要担心Resize导致的图片失真
    1. 除了Resize之外，还有归一化、明暗对比度、裁剪、旋转四种常用数据处理方案。其中，本模型中大部分拐角坐标，或者分割结果，均保持内部为分割文档，外围是背景内容。如果使用分割进行数据增强，则会训练模型学习到一些完全由文档部分组成的信息。这对拟合是不利的，训练结果同样应证了这一情况。
    
- 损失选择
    0. 讨论分割任务常用的两种损失，CrossEntropy和DiceLoss，使用任意损失均可以获得高于回归模型5%的分数。但是两者相比，DiceLoss能够在相同模型的基础下比CrossEntropy提升1-2个百分点成绩。

- 其他内容

    0. 参考.yaml配置信息

- Checkpoint

    0. 提交使用的模型为OCRnet，模型参数保存在submit文件夹中，请勿覆盖。

# 代码部分

## 拉取数据/包

In [ ]:
%cd ~
! wget https://staticsns.cdn.bcebos.com/amis/2022-4/1649731549425/train_datasets_document_detection_0411.zip
! unzip -oq /home/aistudio/train_datasets_document_detection_0411.zip
! rm -rf __MACOSX
! rm -rf /home/aistudio/train_datasets_document_detection_0411.zip

In [43]:
# 将拉取的数据转化为PaddleSeg需要的形式
%cd ~
import cv2
import os
for item in os.listdir('train_datasets_document_detection_0411/segments'):
    img_dir = 'train_datasets_document_detection_0411/segments/'+item
    img = cv2.imread(img_dir)
    img = img/255
    cv2.imwrite(img_dir,img[:,:,1])

/home/aistudio


In [ ]:
# clone PaddleSeg的项目
# ! git clone https://github.com/PaddlePaddle/PaddleSeg.git
! git clone https://gitee.com/PaddlePaddle/PaddleSeg.git

In [ ]:
! pip install paddleseg

## 测试paddleseg

In [ ]:
# test
%cd PaddleSeg/
! python predict.py \
       --config configs/quick_start/pp_liteseg_optic_disc_512x512_1k.yml \
       --model_path https://paddleseg.bj.bcebos.com/dygraph/optic_disc/pp_liteseg_optic_disc_512x512_1k/model.pdparams\
       --image_path docs/images/optic_test_image.jpg \
       --save_dir output/result

## 训练
### 划分数据集

In [ ]:
%cd ~/PaddleSeg/
! python tools/split_dataset_list.py /home/aistudio/ \
    train_datasets_document_detection_0411/images \
    train_datasets_document_detection_0411/segments

### 训练

进行配置，文件路径为 'configs/quick_start/my_model2.yml',内容如下所示：

```
batch_size: 16  #设定batch_size的值即为迭代一次送入网络的图片数量，一般显卡显存越大，batch_size的值可以越大
iters: 120000    #模型迭代的次数

train_dataset:  #训练数据设置
  type: Dataset #数据集名字
  dataset_root: /home/aistudio/ #数据集路径
  train_path: /home/aistudio/train.txt  #数据集中用于训练的标识文件
  num_classes: 2  #指定目标的类别个数（背景也算为一类）
  mode: train #表示用于训练
  transforms: #数据预处理/增强的方式
    - type: RandomHorizontalFlip  #采用水平反转的方式进行数据增强
    - type: RandomDistort #亮度、对比度、饱和度随机变动
      brightness_range: 0.5
      contrast_range: 0.5
      saturation_range: 0.5
    - type: Resize
      target_size: [512, 512]
    - type: Normalize

val_dataset:  #验证数据设置
  type: Dataset #数据集名字
  dataset_root: /home/aistudio/ #数据集路径
  val_path: /home/aistudio/val.txt  #数据集中用于验证的标识文件
  num_classes: 2  #指定目标的类别个数（背景也算为一类）
  mode: val #表示用于验证
  transforms: #数据预处理/增强的方式
    - type: Resize
      target_size: [512, 512]
    - type: Normalize

optimizer: #设定优化器的类型
  type: sgd #采用SGD（Stochastic Gradient Descent）随机梯度下降方法为优化器
  momentum: 0.5 #动量

lr_scheduler: # 学习率的相关设置
  type: PolynomialDecay # 一种学习率类型。共支持12种策略
  learning_rate: 0.0005
  power: 0.5
  end_lr: 0

loss:
  types:
    - type: DiceLoss
    - type: DiceLoss
  coef: [1, 0.4]

model:
  type: OCRNet
  backbone:
    type: HRNet_W18
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w18_ssld.tar.gz
  backbone_indices: [0]
```

In [ ]:
import shutil
shutil.copy('/home/aistudio/my_model2.yml','/home/aistudio/PaddleSeg/configs/quick_start/my_model2.yml')

In [ ]:
# 导出模型
%cd ~/PaddleSeg/
! export CUDA_VISIBLE_DEVICES=0
! python train.py \
       --config configs/quick_start/my_model2.yml \
       --save_interval 10000 \
       --resume_model output/iter_100000 \
       --save_dir output

In [ ]:
%cd ~/PaddleSeg/
# 设置1张可用的卡
! export CUDA_VISIBLE_DEVICES=0
# windows下请执行以下命令
# set CUDA_VISIBLE_DEVICES=0
! python export.py \
       --config configs/quick_start/my_model.yml \
       --model_path output/iter_120000/model.pdparams \
       --save_dir output \
       --input_shape 1 3 512 512

## 测试导出模型

In [ ]:
# 测试模型是否能正常读取
%cd ~
import paddle
path = "PaddleSeg/output/model"
loaded_layer = paddle.jit.load(path)
# inference
loaded_layer.eval()
x = paddle.randn([1, 3, 512, 512], 'float32')
pred = loaded_layer(x)

In [5]:
# 测试模型是否能正常预测，预测内容写入test.jpg

import cv2

img_dir = 'train_datasets_document_detection_0411/images/image_00001.jpg'
img = cv2.imread(img_dir)

mean = [127.5, 127.5, 127.5]
std = [127.5, 127.5, 127.5]

img = paddle.vision.transforms.resize(img, (512, 512))
img = paddle.vision.transforms.normalize(img, mean=mean, std=std, data_format='HWC')

img = img.transpose([2,0,1])

pred = loaded_layer(paddle.to_tensor(img).reshape([1,3,512,512]))
pre = (pred.astype('float32').numpy()*255).transpose([1,2,0])
cv2.imwrite('test.jpg', pre)

## 打包

In [39]:
%cd ~
import shutil
! rm -rf submit
! mkdir submit
shutil.copy('predict.py','submit/predict.py')
shutil.copy('PaddleSeg/output/deploy.yaml','submit/deploy.yaml')
shutil.copy('PaddleSeg/output/model.pdiparams','submit/model.pdiparams')
shutil.copy('PaddleSeg/output/model.pdiparams.info','submit/model.pdiparams.info')
shutil.copy('PaddleSeg/output/model.pdmodel','submit/model.pdmodel')

/home/aistudio


'submit/model.pdmodel'

In [ ]:
# 压缩需要提交的内容，需要提交的文件位于~/submit/

%cd ~/submit/
! rm -rf submit.zip
! zip submit.zip *

## 测试
想要看结果可以直接运行predict.py，需要更替测试集可以在官网下载对应的test数据，并且替换下面cell中的 '/home/aistudio/train_datasets_document_detection_0411/images' 为对应测试图片路径

In [ ]:
%cd ~/submit/
! python predict.py /home/aistudio/train_datasets_document_detection_0411/images /home/aistudio/result